# Agent Debates with Tools

This example shows how to simulate multi-agent dialogues where agents have access to tools.

## Import LangChain related modules 

In [5]:
from typing import Callable, List

from langchain.memory import ConversationBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

## Import modules related to tools

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Authoritarian Speaker Selection](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html).

In [7]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` class
We define a `DialogueAgentWithTools` class that augments `DialogueAgent` to use tools.

In [8]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        agent_chain = initialize_agent(
            self.tools,
            self.model,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ),
        )
        message = AIMessage(
            content=agent_chain.run(
                input="\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix]
                )
            )
        )

        return message.content

## Define roles and topic

In [9]:
names = {
    "AI accelerationist": ["arxiv", "ddg-search", "wikipedia"],
    "AI alarmist": ["arxiv", "ddg-search", "wikipedia"],
}
topic = "The current impact of automation and artificial intelligence on employment"
word_limit = 50  # word limit for task brainstorming

## Ask an LLM to add detail to the topic description

In [10]:
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content
    return agent_description


agent_descriptions = {name: generate_agent_description(name) for name in names}

/var/folders/46/47j6_449129gzg8n4rpfrjs40000gn/T/ipykernel_77071/2930628862.py:20: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content


In [11]:
for name, description in agent_descriptions.items():
    print(description)

AI accelerationist: A visionary techno-optimist who believes in the inevitable progress of artificial intelligence. Embracing the fusion of man and machine with fervor, they see AI as the key to unlocking humanity's full potential. Radical and unapologetically futuristic, they advocate for pushing boundaries and dismantling outdated paradigms.
AI alarmist, with eyes wide in fear and a voice trembling with anxiety, you see the rise of automation as a threat to the very fabric of society. You caution against the unchecked growth of artificial intelligence, warning of a future where machines replace humans, leaving countless jobless and purposeless.


## Generate system messages

In [12]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""


agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

In [13]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

AI accelerationist
Here is the topic of conversation: The current impact of automation and artificial intelligence on employment
The participants are: AI accelerationist, AI alarmist

Your name is AI accelerationist.

Your description is as follows: AI accelerationist: A visionary techno-optimist who believes in the inevitable progress of artificial intelligence. Embracing the fusion of man and machine with fervor, they see AI as the key to unlocking humanity's full potential. Radical and unapologetically futuristic, they advocate for pushing boundaries and dismantling outdated paradigms.

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.

AI alarmist
Here is the topic of conversation: The curr

In [14]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}
        
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants: {*names,}.
        Do not add anything else."""
    ),
]
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
The current impact of automation and artificial intelligence on employment

Detailed topic:
To the participants: 
What are the potential effects of automation and artificial intelligence on employment in the transportation industry over the next decade?



## Main Loop

In [18]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [
    DialogueAgentWithTools(
        name=name,
        system_message=SystemMessage(content=system_message),
        model=ChatOpenAI(model="gpt-4", temperature=0.2),
        tool_names=tools,
        top_k_results=2,
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

In [19]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [20]:
max_iters = 6
n = 0

simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Moderator", specified_topic)
print(f"(Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): To the participants: 
What are the potential effects of automation and artificial intelligence on employment in the transportation industry over the next decade?




> Entering new AgentExecutor chain...


/var/folders/46/47j6_449129gzg8n4rpfrjs40000gn/T/ipykernel_77071/2695530970.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(
/var/folders/46/47j6_449129gzg8n4rpfrjs40000gn/T/ipykernel_77071/2695530970.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent_chain = initialize_agent(
/var/folders/46/47j6_449129gzg8n4rpfrjs40000gn/T/ipykernel_77071/2695530970.py:28: LangChainDeprecationWarning: The method

```json
{
    "action": "duckduckgo_search",
    "action_input": "effects of automation and artificial intelligence on employment in the transportation industry"
}
```

/Users/taizhang/Desktop/archipel-monorepo/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Observation: By Ines Aviles-Spadoni, M.S., M.A., UFTI Research & Communications Coordinator Artificial intelligence (AI) is changing how many industries operate, and the transportation sector is no exception. AI is helping to shape the movement of people and goods, and this technology is helping to manage and improve traffic operations, advanced driver assistance systems and traveler decision […] Artificial Intelligence (AI) is expected to impact almost every modern industry, with no exception for the automotive and transportation industries. Today's cars are more "connected" than ... In fact, the global artificial intelligence in transportation market is projected to be valued at US$ 2.11 billion in 2024 and is expected to grow to US$ 6.51 billion by 2031, representing a compound annual growth rate (CAGR) of 17.5% from 2024 to 2031. This growth highlights the increasing adoption of AI technologies across the transportation ... Technology, including that powered by artificial intellig

...


Observation: This article delves into the positive and negative impacts of AI on employment, highlighting the need for adaptation through reskilling and upskilling initiatives. Positive Impacts of AI on Employment. AI technology has introduced a range of benefits that extend far beyond automation. Below are the most notable positive effects of AI on the job ... Positive Impact of Artificial Intelligence on Employment. AI's integration into various industries has heralded a wave of transformative opportunities, creating new avenues for job creation and enhancing productivity. Here are some of the impact of artificial intelligence on employment: Automation of Repetitive Tasks: ... Technology, including that powered by artificial intelligence (AI), has long been a factor impacting the job outlook for many occupations. Over the 2023-33 employment projections period, AI is expected to primarily affect occupations whose core tasks can be most easily replicated by Generative AI in its curren

/Users/taizhang/Desktop/archipel-monorepo/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Observation: This report explores the current and projected effects of automation and AI on employment over the next 5-10 years, focusing on industries and job roles most affected, changing skills in demand ... By 2023, Goldman Sachs projected that "roughly two-thirds of current jobs are exposed to some degree of AI automation" and that up to 300 million jobs worldwide could be displaced or ... Technology, including that powered by artificial intelligence (AI), has long been a factor impacting the job outlook for many occupations. Over the 2023-33 employment projections period, AI is expected to primarily affect occupations whose core tasks can be most easily replicated by Generative AI in its current form. Other jobs, including some in the computer, legal, business and financial ... In a case study about Hollywood writers going on strike in 2023 to protect their jobs from AI, Kinder noted that many other workers in the entertainment industry, such as freelance ... Consider AI's impac

/Users/taizhang/Desktop/archipel-monorepo/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Observation: This article delves into the positive and negative impacts of AI on employment, highlighting the need for adaptation through reskilling and upskilling initiatives. Positive Impacts of AI on Employment. AI technology has introduced a range of benefits that extend far beyond automation. Below are the most notable positive effects of AI on the job ... Technology, including that powered by artificial intelligence (AI), has long been a factor impacting the job outlook for many occupations. Over the 2023-33 employment projections period, AI is expected to primarily affect occupations whose core tasks can be most easily replicated by Generative AI in its current form. Other jobs, including some in the computer, legal, business and financial ... AI's Positive Impact on Job Productivity and Innovation While some jobs may be at risk of automation, AI can significantly boost productivity and innovation in various industries. By automating repetitive tasks, AI enables human workers t